In [14]:
import glob
import numpy as np

from itertools import groupby

ans_path = './ner-mturk/answers*'
ans_file_list = sorted(glob.glob(ans_path))
single_ans_list = sorted(glob.glob('ner-mturk/single_ans*'))
truth_file = './ner-mturk/ground_truth.txt'
path_w = './MBEM_outputs'
alltask_num = 5985
train_num = 5385
train_num5 = 1077
test_num = 600
redundancy = 5
iteration_times = 5
ans_file = ans_file_list[5]

In [4]:
def read_output(path, class_dic_r, to_label=False):
    f = open(path)
    make_xypair = lambda v: list(zip(*[l.strip().split() for l in v]))
    is_emptyline = lambda x: x.strip() != '' and not x.startswith('-DOCSTART-')
    list_int = lambda t: list([class_dic_r[int(v)] for v in t])
    xypairs = [make_xypair(v) for k, v in groupby(f, is_emptyline) if k]
    if to_label == True:
        return ([p[0] for p in xypairs], [list(list_int(p[1])) for p in xypairs])
    if len(xypairs[0]) == 2:
        return ([p[0] for p in xypairs], [list(p[1]) for p in xypairs])
    else:
        return ([p[0] for p in xypairs], [list(p[1:]) for p in xypairs])

In [6]:
def load_crowd_data(ans_file, truth_file, train_num):
    #answers_list[task_num] = [[words, ...], [worker], [[answers]...]]
    answers_list = []
    sentence = []
    answers = []
    worker = []
    words_dic = {}
    get_worker = lambda x: tuple([i for i, l in enumerate(x) if l !='?'])
    for l in open(ans_file):
        ans = l.split()
        if len(l) < 2:
            answers_list.append([sentence, worker, answers])
            sentence = []
            worker = []
            answers = []
        else:
            if len(worker) < 1:
                worker = get_worker(ans[1:])
            if not ans[0] in words_dic:
                words_dic[ans[0]] = len(words_dic)
            sentence.append(ans[0])
            answers.append(get_ans(worker, ans[1:]))            
    #truth_list: [(task_word, answer), ....]
    truth_list = [tuple(l.split()) for l in open(truth_file) if l != "\n"]
    ans_train = answers_list[0:train_num]
#    ans_test = answers_list[train_num:]
    return ans_train, words_dic

In [8]:
def get_ans(workers, answers):
    ans_label = []
    for a in workers:
        ans_label.append(class_dic[answers[a]])
    return ans_label

In [13]:
class_list = list(set(l.split()[1] for l in open(truth_file) if l != "\n"))
class_dic = {k:i for i, k in enumerate(sorted(class_list))}
class_dic_r = {class_dic[c]:c for c in class_dic}
class_label = {'LOC': 0, 'MISC': 1, 'ORG': 2, 'PER': 3, 'O': 4}
labelnum_dic = {'B-LOC': 0, 'B-MISC': 1, 'B-ORG': 2, 'B-PER': 3, 'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
#データの読み込み
ans_train, words_dic = load_crowd_data(ans_file, truth_file, train_num5)
X_true, y_true = read_output('../MBEM/ner-mturk/ground_truth.txt', class_dic_r)
X_true5, y_true5 = read_output('../MBEM/ner-mturk/testset5.txt', class_dic_r)
workers_count = []
for i in range(len(ans_train)):
    for w in ans_train[i][1]:
        workers_count.append(w)
worker_num = len(set(workers_count))
class_num = len(class_dic)
#多分データを変えるごとにクラス確認したくなるので残しておく
num_words = 0
for l in ans_train:
    num_words += len(l[0])
print("worker_num", worker_num)
print("クラス：",class_dic)
print("クラス数：", class_num)
print("タスク数",len(ans_train))
print("タスク数(単語):", num_words)
print("語彙数", len(words_dic))

worker_num 46
クラス： {'B-LOC': 0, 'B-MISC': 1, 'B-ORG': 2, 'B-PER': 3, 'I-LOC': 4, 'I-MISC': 5, 'I-ORG': 6, 'I-PER': 7, 'O': 8}
クラス数： 9
タスク数 1077
タスク数(単語): 14119
語彙数 6450


In [ ]:
#input 
#ラベル :c
#単語: x

K = worker_num
J = class_num
#初期値を設定する
conf = np.zeros((K,J,J,J)) #worker,true_label,woker_ans(n-1),worker_ans(n)
rho = np.zeros((J)) #出力確率
trans_prob = np.zeros((J,J))

#前向き後ろ向きアルゴリズム
#論文の7式

#ビタビアルゴリズム

#

In [ ]:
s_symbol = [ "w1", "w2", "w3" ] # 状態
c = len(s_symbol)               # 状態数
v_symbol = [ "v1", "v2" ]       # 出力記号
m = len(v_symbol)               # 出力数

# 遷移確率行列
A = np.array([[0.1, 0.7, 0.2], [0.2, 0.1, 0.7], [0.7, 0.2, 0.1]])
# 出力確率行列
B = np.array([[0.9, 0.1], [0.6, 0.4], [0.1, 0.9]])
# 初期確率行列
p = np.array([1/3, 1/3, 1/3])

class HMM:

    def __init__(self, p, A, B): 
        self.p = p
        self.A = A
        self.B = B

    def forward(self, x, c):
        n = len(x) # 観測回数

        # 前向きアルゴリズム
        alpha = np.zeros((n, c))

        # Step1
        alpha[0, :] = p[:] * B[:, x[0]]

        # Step2
        for t in range(1, n):
            alpha[t, :] = np.dot(alpha[t-1, :], A) * B[:, x[t]]

        # Step3
        return round(np.sum(alpha[-1]), 3)

    def backward(self, x, c):
        n = len(x) # 観測回数

        # 後ろ向きアルゴリズム
        # Step1
        beta = np.zeros((n, c))
        beta[-1, :] = 1

        # Step2
        for t in range((n-1), 0, -1):
            beta[t-1, :] = np.dot(self.A, (self.B[:, x[t]] * beta[t, :]))

        # Step3
        return round(sum(p[:] * self.B[:, x[0]] * beta[0, :]), 3)

if __name__=="__main__":
    # 観測結果
    x = [ 0, 1, 0 ] # v1, v2, v1という結果

    hmm = HMM(p, A, B)
    print(hmm.forward(x, c))
    print(hmm.backward(x, c))

In [ ]:
def post_prob_DS(est_labels, ans_train, k, worker_num, num_words):
    #学習機とラベルの更新
    n = len(ans_train)
    m = worker_num
    e_conf = np.zeros((m,k,k,k))
    temp_conf = (1/float(k))*np.ones((m,k,k,k))
    labels_md = 0.000001 * np.ones((k,k)) #正規化の時に困りそうなので0にしないでおく
    task_acc = np.zeros((n, k, k))
    #est_labelsを正解ラベルとして混同行列を作成 式(7)
    for i in range(n):
        pre_ans = ans_train[i][2][0]
        for l in range(len(ans_train[i][2])-1):
            l += 1
            #真のラベルの周辺分布を更新
            labels_md[int(est_labels[i][l-1]),int(est_labels[i][l])] += 1
            for a, j in enumerate(ans_train[i][1]): #各回答ワーカーについて
                temp_conf[j,int(est_labels[i][l]),:,:] = temp_conf[j,int(est_labels[i][l]),:,:] + np.outer(np.eye(k)[ans_train[i][2][l-1][a]],np.eye(k)[ans_train[i][2][l][a]]) #外積 混同行列の作成
    #temp_confの正規化
    for j in range(m):  
        for p in range(k):
            e_conf[j,p,:,:] = np.divide(temp_conf[j,p,:,:],np.outer(np.sum(temp_conf[j,p,:,:],axis =1),np.ones(k)))
    labels_md = np.divide(labels_md, np.sum(labels_md)*np.ones(k))
    #混同行列を使って真のラベル推定(論文p5の(5)式)
    for i in range(n):
        for l in range(len(ans_train[i][2])):
            temp_class = 0.0
            for r, a in enumerate(ans_train[i][1]):
                if l == 0:
                    w_conf = np.sum(e_conf[a,:,:,:], axis=1)
                else:
                    w_conf = e_conf[a,:,ans_train[i][2][l-1][r],:]
                #混同行列の該当列をとる
                temp_acc = np.log(np.dot(w_conf,np.transpose(np.eye(k)[ans_train[i][2][l][r]])))
                temp_class = temp_class + temp_acc
            temp_class = np.log(labels_md) +temp_class
            est_labels[i][l] = np.argmax(temp_class)%9
    return est_labels, e_conf